In [1]:
import sys
import pymysql
pckg_path = "C:/Users/22668/Desktop"
sys.path.append(pckg_path)
from jbot.config.DatabaseConfig import *
from jbot.config.PathConfig import *
from jbot.utils.Preprocess import Preprocess
import pandas as pd
import numpy as np
PathConfig()

raw_chatbot_train_path1 = C:/Users/22668/Desktop/jbot/train_tools/qna/교내학식.xlsx
raw_chatbot_train_path2 =  C:/Users/22668/Desktop/jbot/train_tools/qna/교내공지.xlsx
raw_chatbot_train_path3 = C:/Users/22668/Desktop/jbot/train_tools/qna/위치.csv
raw_chatbot_train_path4 = C:/Users/22668/Desktop/jbot/train_tools/qna/sns.xlsx
raw_chatbot_train_path5 = C:/Users/22668/Desktop/jbot/train_tools/qna/과사무실.csv
raw_chatbot_train_path6 = C:/Users/22668/Desktop/jbot/train_tools/qna/교내식당 운영시간.xlsx
train_file = C:/Users/22668/Desktop/jbot/train_tools/qna/train_data.xlsx // 챗봇학습용(답변용)데이터
user_dic_path = C:/Users/22668/Desktop/jbot/utils/jbot_dict.tsv // 사용자정의사전
corpus_path = C:/Users/22668/Desktop/jbot/train_tools/dict/corpus.xlsx // 의도분류모델(딥러닝모델)학습용데이터
chatbot_dict_path = C:/Users/22668/Desktop/jbot/train_tools/dict/chatbot_dict.bin // 단어-정수 매핑데이터(pytorch의 lookuptable 또는 word2index)
intent_model_path = C:/Users/22668/Desktop/jbot/models/intent/intent_model.h5 // 학습된 의도분류모델
intent_mapping_path = C:/Users/2266

In [2]:
def create_ner_data():
    df1 = pd.read_excel(ner_data_path1)
    #df2 = pd.read_csv(ner_data_path2) #공지사항은 개체분류가 필요하지 않으므로 pass
    df3 = pd.read_excel(ner_data_path3)
    df4 = pd.read_excel(ner_data_path4)
    df5 = pd.read_excel(ner_data_path5)
    df6 = pd.read_excel(ner_data_path6) 

    df = pd.concat([df1,df3,df4,df5,df6],axis=0)
    #df = df.drop_duplicates()
    #print(df)
    
    data = []
    for row in df.iterrows():
        inform = row[1]
        var_num = len(row[1])
        for i in range(var_num):
            #print(i)
            #print(inform[i])
            if i == 0:
                intent = inform[i]
            elif i == 1:
                ner = inform[i]
            else:
                synonym = inform[i]
                data.append((intent,ner,synonym))
    _df = pd.DataFrame(data)
    _df.columns = ["의도","개체명","유의어"]
    _df = _df.dropna(subset = ["유의어"])
    _df.to_excel(ner_final)
    return _df
    
_df = create_ner_data()

In [3]:
_df.의도.value_counts()

학과사무실        356
위치           174
교내학식          50
교내식당 운영시간     34
SNS안내         27
Name: 의도, dtype: int64

In [4]:
def add_depart():
    개체명인식final = pd.read_excel(ner_final).drop(columns = "Unnamed: 0")
    과사무실원본 = pd.read_csv(raw_chatbot_train_path5)
    cond = 개체명인식final["의도"] == "학과사무실"
    개체명인식과목록 = 개체명인식final[cond]


    전체과 = 과사무실원본["개체"].unique()
    인식가능한과 = 개체명인식과목록["개체명"].unique()
    add_data = []
    for 과 in 전체과:
        if 과 not in 인식가능한과:
            add_raw = ["학과사무실",과,과]
            add_data.append(add_raw)
    add_df = pd.DataFrame(add_data)
    add_df.columns = ["의도","개체명","유의어"]
    _df = pd.concat([개체명인식final,add_df],axis=0).reset_index(drop = True)
    for var_name,values in _df.iteritems():
        lower_values = []
        for value in values:
            lower_value = value.lower()
            lower_values.append(lower_value)
        _df[var_name] = lower_values
    return _df

In [5]:
_df = add_depart()
_df.to_excel(ner_final)